In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import imgaug as ia
import numpy as np
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from collections import defaultdict
from PIL import Image
from project.utils import TRAIN_IMAGES_PATHS, TRAIN_LABELS_PATHS, SEED, parse_label

In [9]:
ia.seed(SEED)
len(TRAIN_IMAGES_PATHS), len(TRAIN_LABELS_PATHS)

(5921, 5921)

In [4]:
ia_ds = defaultdict(list)
for label_path, img_path in zip(TRAIN_LABELS_PATHS, TRAIN_IMAGES_PATHS):
    img = Image.open(img_path)
    label = parse_label(label_path)
    
    klass, *bbox = label
    img_w, img_h = img.size
    x, y = bbox[0] * img_w, bbox[1] * img_h
    w, h = bbox[2] * img_w, bbox[3] * img_h
    x1, x2 = x - (w // 2), x + (w // 2)
    y1, y2 = y + (h // 2), y - (h // 2)
    bb = BoundingBox(x1=x1, x2=x2, y1=y1, y2=y2, label=klass)
    ia_ds[klass].append((img_path, img, label_path, BoundingBoxesOnImage([bb], shape=(img_h, img_w))))

In [5]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Crop(percent=(0, 0.1)),
    iaa.Sometimes(
        0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    iaa.LinearContrast((0.75, 1.5)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-180, 180),
        shear=(-8, 8)
    )
], random_order=True)

In [13]:
def augment_image(image: np.array, bbox: BoundingBox, n: int) -> list[tuple[np.array, BoundingBox]]:
    output = []
    for _ in range(n):
        while True:
            image_aug, bbox_aug = seq(image=image, bounding_boxes=bbox)
            if len(bbox_aug.remove_out_of_image()) == 0:
                continue
            output.append((bbox_aug.clip_out_of_image()[0], image_aug))
            break
    return output

In [14]:
print({k: len(v) for k, v in ia_ds.items()})
classes_to_augment = [(0, 20), (2, 20), (4, 4), (5, 5)]

{1: 2620, 3: 2627, 5: 237, 0: 28, 4: 362, 2: 47}


In [15]:
for klass, n in classes_to_augment:
    for img_path, img, label_path, label in ia_ds[klass]:
        augmented = augment_image(np.array(img), label, n)
        for idx, (aug_label, aug_img) in enumerate(augmented):
            aug_img_path = img_path.with_name(img_path.stem + f'_{idx}' + img_path.suffix)
            aug_label_path = label_path.with_name(label_path.stem + f'_{idx}' + label_path.suffix)

            aug_img = Image.fromarray(aug_img)
            aug_img.save(aug_img_path)
            
            with open(aug_label_path, "w") as f:
                xc, yc = (aug_label.x1 + aug_label.x2) // 2, (aug_label.y1 + aug_label.y2) // 2
                xc, yc = xc / img_w, yc / img_h
                w, h = (aug_label.x2 - aug_label.x1) / img_w, (aug_label.y2 - aug_label.y1) / img_h
                string = " ".join(map(str, [aug_label.label, xc, yc, w, h]))
                f.write(string)